In [1]:
!pip install torchtext==0.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Not uninstalling torch at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'torch'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import torch

SEED = 17
torch.manual_seed(SEED)

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def set_device():
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

DEVICE = set_device()

# Veri okuma kısmı (hazır)

In [4]:
import torch
import torchtext.legacy.data as data
import torchtext.legacy.datasets as datasets
import random
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('webtext')

SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)

def load_dataset(sentence_length=50, batch_size=32, seed=SEED):
    TEXT = data.Field(sequential=True,
                      tokenize=nltk.word_tokenize,
                      lower=True,
                      include_lengths=True,
                      batch_first=True,
                      fix_length=sentence_length)
    LABEL = data.LabelField(dtype=torch.float)

    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    
    # Veri kümesini dilimle
    train_data, val_data = train_data.split(split_ratio=0.8, random_state=random.seed(seed))
    test_data = test_data[:500]

    # Vocab'ları oluşturun
    TEXT.build_vocab(train_data, max_size=10000)
    LABEL.build_vocab(train_data)
    
    # DataLoader'ları oluşturun
    train_loader, val_loader, test_loader = data.BucketIterator.splits(
        (train_data, val_data, test_data),
        batch_size=batch_size,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True
    )

    return train_loader, val_loader, test_loader, TEXT

train_loader, val_loader, test_loader, TEXT = load_dataset(sentence_length=50, batch_size=32, seed=SEED)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.


In [5]:
def text_from_dict(arr, dictionary):
  text = []
  for element in arr:
    text.append(dictionary[element])
  return text


def view_data(TEXT, train_iter):
  for idx, batch in enumerate(train_iter):
    text = batch.text[0]
    target = batch.label

    for itr in range(25, 30):
      print('Review: ', ' '.join(text_from_dict(text[itr], TEXT.vocab.itos)))
      print('Label: ', int(target[itr].item()), '\n')

    print('[0: Negative Review, 1: Positive Review]')
    if idx==0:
      break

# view_data'yı çağır, training set'teki verilerin az bir kısmını görüntüle/print et
view_data(TEXT, train_loader)

Review:  in all my 60 years of age , i have learned that when we watch a movie there is an <unk> ( whether we want it or not ) <unk> with an specific character. < br / > < br / > sometimes because the character <unk> certain gesture ,
Label:  1 

Review:  at first glance , it would seem natural to compare where the sidewalk ends with laura . both have <unk> qualities , both were directed by otto preminger , and both star dana andrews and gene tierney . but that 's where most of the comparisons end . laura dealt
Label:  1 

Review:  there 's more to offer in the opening of the odd couple than in the entirety of most films . felix <unk> ( the poor guy 's <unk> even <unk> him ) checks into a new york hotel . a cleaning lady says `` good night . '' `` goodbye
Label:  1 

Review:  `` the invisible ray '' is part science fiction and part horror . it was also the third of seven <unk> features . in this entry the <unk> role goes to boris <unk> < br / > < br / > through specially designed <unk>

# Bizim kodlarımız (our NN code)

In [6]:
import torch
import torch.nn as nn

# dropout ekleyebilir misiniz? her layer arasına, az miktar
class MLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden1, hidden2, hidden3, num_classes):
        super(MLP, self).__init__()

        self.drop = nn.Dropout(p=0.1)
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # bu kısım kelimeleri özniteliklere çevirmek için
        self.fc1 = nn.Linear(embedding_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, hidden3)
        self.fc4 = nn.Linear(hidden3, num_classes)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, text_lengths):
        # alt 3 satırı chatgpt'den aldım
        embedded = self.embedding(text)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        packed_output, _ = nn.utils.rnn.pad_packed_sequence(packed_embedded, batch_first=True)
        
        x = self.drop(self.relu(self.fc1(packed_output[:, -1, :])))
        x = self.drop(self.relu(self.fc2(x)))
        x = self.drop(self.relu(self.fc3(x)))
        output = self.sigmoid(self.fc4(x))
        return output

"""
Veriye uygun bir aktivasyon fonksiyonu olarak, 
genellikle çok sınıflı sınıflandırma problemlerinde tercih edilen 
"Softmax" fonksiyonunu kullanabilirsiniz
    -ChatGPT
"""


'\nVeriye uygun bir aktivasyon fonksiyonu olarak, \ngenellikle çok sınıflı sınıflandırma problemlerinde tercih edilen \n"Softmax" fonksiyonunu kullanabilirsiniz\n    -ChatGPT\n'

In [11]:
# training, earlystop'lu validation kodu
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# train_loader, val_loader, test_loader

vocab_size = len(TEXT.vocab)
embedding_dim = 100
hidden1 = 64
hidden2 = 16
hidden3 = 8

patience = 2

model = MLP(vocab_size, embedding_dim, hidden1, hidden2, hidden3, 1)

# Loss fonksiyonu ve optimizer tanımlama
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # learning rate seçin

batch_size = 32
num_epochs = 100

# Eğitim döngüsü
best_val_loss = 10000
early_stopping_counter = 0

train_losses, val_losses = [], []
for epoch in range(num_epochs):
    model.train()

    avg_train_loss = 0
    for batch in train_loader:
        
        text, text_lengths = batch.text
        # Gradientleri sıfırlama
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(text, text_lengths)
        
        # Loss hesaplama
        loss = criterion(outputs.squeeze(), batch.label)
        
        # Backward pass
        loss.backward()
        
        # Parametre güncelleme
        optimizer.step()

        avg_train_loss += loss

    curr_train_loss = avg_train_loss / len(train_loader)
    train_losses.append(curr_train_loss)

    # Doğrulama (validation) döngüsü
    model.eval()
    avg_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            
            text, text_lengths = batch.text
            # Forward pass
            val_outputs = model(text, text_lengths)
            
            # Loss hesaplama
            val_loss = criterion(val_outputs.squeeze(), batch.label)

            avg_val_loss += loss

    curr_val_loss = avg_val_loss / len(val_loader)
    val_losses.append(curr_val_loss)

    # En iyi doğruluk skorunu kontrol et - Accuracy değil loss'a göre değiştir
    if curr_val_loss < best_val_loss:
        best_val_loss = curr_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        print("Earlystop counter ", early_stopping_counter)
        
    # Early stopping kontrolü
    if early_stopping_counter >= patience:
            print("Early stopping!")
            break
        
    # Ekrana yazdırma
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {curr_train_loss:.4f}, Val Loss: {curr_val_loss:.4f}")




Epoch [1/100], Train Loss: 0.6934, Val Loss: 0.6927
Epoch [2/100], Train Loss: 0.6924, Val Loss: 0.6925
Epoch [3/100], Train Loss: 0.6895, Val Loss: 0.6613
Earlystop counter  1
Epoch [4/100], Train Loss: 0.6818, Val Loss: 0.6997
Earlystop counter  2
Early stopping!


In [ ]:
# test setinden sonuç alan kodu - bunu da siz yazarsınız artık :D



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=700e962d-3fcf-4c82-8c8d-f1f2cc26fdf7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>